Ковшик А.С., 201-361, вариант 5


# Математические методы обработки данных

## Проверка статистических гипотез

### Лабораторная работа №3

1. По расчетам лабораторной работы №1 проверить гипотезу соответствия нормальному закону распределения на уровне значимости 0,1.
2. Решить самостоятельно следующие задачи (согласно варианту):
   Вариант 5. Для сравнения организации работы на двух однотипных предприятиях, были взяты выборочные данные объемами n1 и n2 соответственно по признаку – объемы выпущенной продукции в у.е. Оценки дисперсии $S^2_1$ и $S^2_2$ даны ниже. Можно ли считать, что предприятия работают одинаково точно. Уровень значимости выбрать самостоятельно.
   
   $n_1 = 11, n_2 = 9; S^2_1 = 83.6; S^2_2 = 74.8; \alpha = 0.1$
   

<hr>

#### 1. Проверка статистической гипотезы

Для проверки основной гипотезы: $H_0 : X = N(0.1;1.082)$ воспользуемся критерием согласия Пирсона. В качестве меры расхождения
эмпирического и теоретического законов принимается величина
$$ \chi^2 = \sum\limits^{v}_{i=1}{\frac{(n_i-n*p_i^{теор})^2}{n*p_i^{теор}}}, $$
которая имеет $\chi^2$ распределение с числом степеней свободы $k=v-l-1$, где $v$ - число (новое) групп вариационного ряда; $l$ – число неизвестных параметров предполагаемой модели, оцениваемых по результатам наблюдений (если все параметры предполагаемого закона известны точно, то $l=0$).

Для вычисления критерия $\chi^2$ заполним вспомогательную расчетную таблицу (DataFrame).


In [17]:
import numpy as np
import pandas as pd
import scipy.stats as sts
import matplotlib.pyplot as plt


In [18]:
dataStr = "0,414 0,011 0,666 -1,132 -0,410 -1,077 1,484 -0,340 0,789 -0,494 0,364 -1,237 -0,044 -0,111 -0,210 0,931 0,616 -0,377 -0,433 1,048 -0,037 0,759 0,609 -2,043 -2,290 0,404 -0,543 0,486 0,869 0,347 2,816 -0,464 -0,632 -1,614 0,372 -0,074 -0,916 1,314 -0,038 0,673 0,563 -0,107 0,131 -1,808 0,284 0,458 1,307 -1,625 -0,629 -0,504 -0,0056 -0,131 0,048 1,879 -1,016 0,360 -0,119 2,331 1,672 -1,053 0,840 0,246 -0,237 -1,312 1,603 -0,952 -0,566 1,600 0,465 1,951 0,110 0,251 0,116 -0,957 -0,190 1,479 -0,986 1,249 1,934 0,070 -1,358 -1,246 -0,959 -1,297 -0,722 0,925 0,783 -0,402 0,619 1,826 1,272 -0,945 0,494 0,050 -1,696 1,876 0,063 0,132 0,682 0,544 -0,417 -0,666 -0,104 -0,253 -2,543 -1,133 1,987 0,668 0,360 1,927 1,183 1,211 1,765 0,035 -0,359 0,193 -1,023 -0,222 -0,616 -0,060 -1,319 -0,785 -0,430 -0,298 0,248 -0,088 -1,379 0,295 -0,115 -0,621 -0,618 0,209 0,979 0,906 -0,096 -1,376 1,047 -0,872 -2,200 -1,384 1,425 -0,812 0,748 -1,095"
dataArr = np.array(dataStr.replace(",", ".").split(" ")).astype(float)
df = pd.DataFrame({'Частота': pd.Series(dataArr[19:119])})

interval_range = pd.interval_range(start=-3, end=3)
df['Интервал'] = pd.cut(df['Частота'],
                        bins=interval_range,
                        labels=[1, 2, 3])

df = df.groupby('Интервал').count()
df = pd.DataFrame({'Интервал': df.index.values,
                   'Частота': df['Частота'].values})
df


,Интервал,Частота
0,"(-3, -2]",3
1,"(-2, -1]",12
2,"(-1, 0]",30
3,"(0, 1]",34
4,"(1, 2]",19
5,"(2, 3]",2


Для вычисления вероятностей попадания в интервал $i$ ($p_i^{теор}$) применим формулу и таблицу функции Лапласа
$$ \phi(\frac{x_{i+1}-\overline{x}}{S}) - \phi(\frac{x_i-\overline{x}}{S}), $$
где $S$ = 1.083, $\overline{x}$ = 0.1


In [19]:
from scipy.special import erf
Phi = lambda x: erf(x/2**0.5)/2

mean = 0.1
std = 1.0824588545146516
left = []
right = []
for i, v in enumerate(df['Интервал'].values):
    left.append((v.left - mean) / std)
    right.append((v.right - mean) / std)

df = df.assign(x1=left)
df = df.assign(x2=right)
df = df.assign(Ф_x1=Phi(df.x1))
df = df.assign(Ф_x2=Phi(df.x2))
df = df.assign(Pi=df['Ф_x2'] - df['Ф_x1'])
df = df.assign(Pi100=df['Pi'] * 100)
df = df.assign(K=((df['Частота'] - df['Pi100']) ** (2)) / df['Pi100'])


Критерий согласия можно использовать только в том случае, когда для каждого интервала $n_i^{теор} \geq 5$. Поэтому интервалы, для которых это условие не выполняется, объединим с соседними и число интервалов соответственно уменьшим с 6 до 4.


In [20]:
df.loc[0, 'K'] = df.loc[5, 'K'] = np.NAN
df.loc[1, 'K'] = (((df.loc[0, 'Частота'] + df.loc[1, 'Частота']) - (df.loc[0, 'Pi100'] +
                  df.loc[1, 'Pi100'])) ** (2)) / (df.loc[0, 'Pi100'] + df.loc[1, 'Pi100'])
df.loc[4, 'K'] = (((df.loc[4, 'Частота'] + df.loc[5, 'Частота']) - (df.loc[4, 'Pi100'] +
                  df.loc[5, 'Pi100'])) ** (2)) / (df.loc[4, 'Pi100'] + df.loc[5, 'Pi100'])
print(f"x^2 = {df.K.sum()}")
df


x^2 = 0.09759875269060264


,Интервал,Частота,x1,x2,Ф_x1,Ф_x2,Pi,Pi100,K
0,"(-3, -2]",3,-2.863850,-1.940028,-0.497907,-0.473812,0.024096,2.409554,NaN
1,"(-2, -1]",12,-1.940028,-1.016205,-0.473812,-0.345234,0.128578,12.857774,0.004681
2,"(-1, 0]",30,-1.016205,-0.092382,-0.345234,-0.036803,0.308431,30.843126,0.023048
3,"(0, 1]",34,-0.092382,0.831440,-0.036803,0.297138,0.333940,33.394039,0.010996
4,"(1, 2]",19,0.831440,1.755263,0.297138,0.460393,0.163255,16.325528,0.058875
5,"(2, 3]",2,1.755263,2.679086,0.460393,0.496309,0.035916,3.591599,NaN


В результате расчетов получили значение критерия $\chi^2_{набл} = 0.0976$.

Теперь, определим критическую точку. У предполагаемой нормальной модели были неизвестны оба параметра ($\alpha, \sigma$) и они оценивались по результатам наблюдений, поэтому $l$=2. Тогда

$$
\begin{rcases}
 \alpha = 0,1 & \to \gamma = 1 - \alpha = 0.9 \\
 l = 2, v = 4 & \to k = v - l - 1 = 1
\end{rcases} \to \chi^2_{\gamma;k} = 2.706
$$


In [21]:
sts.chi2.isf(0.1, 1)


2.70554345409542

Таким образом, критическая область есть интервал $(2.706;+\infty)$. Наблюдаемое значение статистики Пирсона не попадает в критическую область: $К_{набл} < K_{кр}$, поэтому нет оснований отвергать основную гипотезу. Справедливо предположение о том, что данные выборки имеют нормальное распределение.


<hr>

### 2. Задача, вариант 5

Для сравнения организации работы на двух однотипных предприятиях, были взяты выборочные данные объемами n1 и n2 соответственно по признаку – объемы выпущенной продукции в у.е. Оценки дисперсии $S^2_1$ и $S^2_2$ даны ниже. Можно ли считать, что предприятия работают одинаково точно. Уровень значимости выбрать самостоятельно.

$n_1 = 11, n_2 = 9; S^2_1 = 83.6; S^2_2 = 74.8; \alpha = 0.1$


Проведем проверку по F-критерию:
$$ F_{наб} = \frac{S^2_Б}{S^2_М} = \frac{83.6}{74.8} = 1.118 $$
Здесь $S^2_Б = 83.6$ является большей из двух оценок дисперсии, а $S^2_М = 74.8$ - является меньшей дисперсией.


$m_1 = n_1 - 1 = 11 - 1 = 10$, т.к. 1 имеет большую дисперсию, $m_2 = n_2 - 1 = 8 - 1 = 7$. Находим при $\alpha = 0,1$
$$ F_{кр}=F(\alpha/2=0.05;10;7)= 3.64 $$
Т.к. $F_{наб.}<F_{кр.}$, то нет основания считать, что точность работы предприятий разная.


In [22]:
sts.f.ppf(q=1-0.05, dfn=10, dfd=7)


3.636523120628346